In [1]:
from google.colab import drive
drive.mount._DEBUG = True
drive.mount('/content/drive', force_remount=True)

unset HISTFILE; export PS1="root@c753ca579e4d-29d2e76a98274a70ac2337a38ab1ca1e: "
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/content# root@c753ca579e4d-29d2e76a98274a70ac2337a38ab1ca1e: umount -f /content/drive || umount /content/drive; pkill -9 -x drive
umount: /content/drive: no mount point specified.
umount: /content/drive: no mount point specified.
root@c753ca579e4d-29d2e76a98274a70ac2337a38ab1ca1e: pkill -9 -f /opt/google/drive/directoryprefetcher_binary
root@c753ca579e4d-29d2e76a98274a70ac2337a38ab1ca1e: ( while `sleep 0.5`; do if [[ -d "/content/drive" && "$(ls -A /content/drive)" != "" ]]; then echo "google.colab.drive MOUNTED"; break; fi; done ) &
[1] 149
root@c753ca579e4d-29d2e76a98274a70ac2337a38ab1ca1e: cat /tmp/tmpyrpxh9uz/drive.fifo | head -1 | ( /opt/google/drive/drive --features=fuse_max_background:1000,max_read_qps:1000,max_write_qps:1000,max_operation_batch_size:15,max_parallel_push_task_instances:1

In [2]:
import os
os.chdir('데이터 있는 위치 경로')

모델 불러오기

In [10]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms


In [5]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=adef3da658ade9e59398960c6f0fa023659573ef4ec2885311ce42ac5c6f6d7f
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [6]:
from efficientnet_pytorch import EfficientNet
model3 = EfficientNet.from_pretrained('efficientnet-b3', num_classes=5) #95

model3.load_state_dict(torch.load('hackaton_effi_b3.pt'))

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b3


<All keys matched successfully>

In [7]:
from efficientnet_pytorch import EfficientNet
model4 = EfficientNet.from_pretrained('efficientnet-b7', num_classes=5) #97

model4.load_state_dict(torch.load('hackaton_effi_b7.pt'))

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth


  0%|          | 0.00/254M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b7


<All keys matched successfully>

In [8]:
import torchvision.models as models

model5=models.resnet101(pretrained=True) #95
num_classes = 5
num_ftrs = model5.fc.in_features
model5.fc = nn.Linear(num_ftrs, num_classes)

model5.load_state_dict(torch.load('hackaton_resnet_101.pt'))

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

<All keys matched successfully>

In [11]:
test_transform = transforms.Compose([transforms.Resize((256,256)),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485 , 0.456, 0.406], [0.229, 0.224 , 0.225])
                                    ])

In [13]:
test_dataset=datasets.ImageFolder('test데이터셋 경로',transform=test_transform)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1,shuffle=False)

In [14]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    
device

'cuda'

In [33]:
from tqdm.auto import tqdm
acc=0

model3.to(device)
model4.to(device)
model5.to(device)
with torch.no_grad(): 

    model3.eval()
    model4.eval()
    model5.eval()
    for data, labels in tqdm(test_dataloader):

        data, labels = data.to(device), labels.to(device)

        logits3 = model3(data)
        ps3 = F.softmax(logits3, dim=1)

        logits4 = model4(data)
        ps4 = F.softmax(logits4, dim=1)

        logits5 = model5(data)
        ps5 = F.softmax(logits5, dim=1)

        ps=(ps3+ps4+ps5)/3

        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.reshape(top_class.shape)
        acc += torch.mean(equals.type(torch.float)).detach().cpu()
        
# calculate average losses
acc /= len(test_dataloader)
print('리멤버 team test accuracy: {:.6f}'.format(acc))


  0%|          | 0/89 [00:00<?, ?it/s]

리멤버 team test accuracy: 0.977528
